INGESTÃO DE DADOS

In [42]:
import pandas as pd

'''
Nada do que esta escrito nessa secao foi descrito na Biblia. Oficialmente fomos abandonados por Deus.

Esse dataset eh grande pra caralho, e ninguem no mundo tem memoria suficiente pra processar tudo de uma vez.
tipo eu to falando de 200GB de memoria RAM, vai se fuder mano, ninguem tem isso.
Detalhe, descobri essa noticia depois de 15min do computador trabalhando. Imagina ficar sentado na frente de uma tela soh olhado
o computador carregar, rezando a Deus que desse certo. Nao deu certo.
primeiramente, eu tentei separar em pedacos, soh usando os chunks e trabalhando com 7 for loops por vez, um pra cada planilha
melhorou, agora agnt precisava de 100GB de memoria RAM. Mesmo assim, vai se fuder 
Depois eu pensei (eu neh, nessa hora a unica pessoa que tinha saco pra fazer essa bosta era o ChatGPT) em fazer um por vez,
cagando para os chunks e so fazendo um atras do outro. Claro, nao funcionou.
Entao, eu, ja de saco cheio com esse dataset de bosta, taquei o fodasse e soh pedi pro Gepeto resolver pra mim o problema
quando ele me manda a mesma MERDA que eu tinha feito, soh que com a porra dos chunks, eu oficialmente desisti dessa bosta
Obrigado a todos pela Compreensao.

PS: Vai se fuder administracao.

'''


# Define chunk size for processing large datasets
chunk_size = 1000

# Define file paths and columns to use
file_paths = {
    'admissions': 'csvs/ADMISSIONS.csv',
    'microbiology_events': 'csvs/MICROBIOLOGYEVENTS.csv',
    'prescriptions': 'csvs/PRESCRIPTIONS.csv',
    'labevents': 'csvs/LABEVENTS.csv',
    'diagnoses_icd': 'csvs/DIAGNOSES_ICD.csv',
    'procedures_icd': 'csvs/PROCEDURES_ICD.csv',
    'patients': 'csvs/PATIENTS.csv',
    'icustays': 'csvs/ICUSTAYS.csv'
}

cols_to_use = {
    'admissions': ['subject_id', 'hadm_id' ,'admittime', 'dischtime', 'ethnicity', 'admission_type'],
    'microbiology_events': ['row_id', 'subject_id', 'hadm_id', 'chartdate', 'charttime', 'spec_itemid', 'spec_type_desc', 
                            'org_itemid', 'org_name', 'isolate_num', 'ab_itemid', 'ab_name', 'dilution_text', 
                            'dilution_comparison', 'dilution_value', 'interpretation'],
    'prescriptions': ['subject_id', "hadm_id", 'drug', 'enddate'],
    'labevents': ['subject_id', 'itemid', 'hadm_id','valuenum', 'valueuom', 'flag'],
    'diagnoses_icd': ['subject_id', 'icd9_code', 'hadm_id', 'seq_num'],
    'procedures_icd': ['subject_id', 'icd9_code',  'hadm_id','seq_num'],
    'patients': ['subject_id', 'gender', 'dob' ,'dod'],
    'icustays': ['subject_id', 'los', 'first_careunit', 'hadm_id' ,'last_careunit']
}

# Specify dtype for memory optimization
dtype_spec = {
    'subject_id': 'Int32',  # Nullable integer type
    'hadm_id': 'Int32',     # Nullable integer type
    'itemid': 'Int32',
    'valuenum': 'float32'    # Floats can have NaN, so no issues here
}

# Helper function to merge two datasets chunk by chunk and write to a file
def merge_and_save(existing_df, new_df_chunks, on='subject_id', suffixes=('', '_new'), output_file='intermediate.csv'):
    """
    Merges an existing DataFrame with a new one (loaded in chunks) and writes the result to a file.
    """
    # Process each chunk of the new DataFrame
    for new_chunk in new_df_chunks:
        # Merge the current chunk with the existing DataFrame
        merged_chunk = pd.merge(existing_df, new_chunk, on=on, how='outer', suffixes=suffixes)
        
        # Save the merged chunk to the output file
        merged_chunk.to_csv(output_file, mode='a', header=False, index=False)

        # Optional: Delete the merged chunk from memory after writing to disk
        del merged_chunk

# Define the output file for the final merged result
output_file = 'PELO_AMOR_DE_DEUS_NAO_DELETA_ESSA_PORRA.csv'

# Step 1: Start with the first dataset (e.g., microbiology_events)
for micro_chunk in pd.read_csv(file_paths['microbiology_events'], usecols=cols_to_use['microbiology_events'], dtype=dtype_spec, chunksize=chunk_size):
    # Save the first chunk to initialize the output file
    micro_chunk.to_csv(output_file, mode='w', header=True, index=False)

    # Step 2: Merge with each subsequent dataset one at a time

    # Merge with prescriptions
    pres_chunks = pd.read_csv(file_paths['prescriptions'], usecols=cols_to_use['prescriptions'], dtype=dtype_spec, chunksize=chunk_size)
    merge_and_save(micro_chunk, pres_chunks, on='subject_id', suffixes=('', '_pres'), output_file=output_file)

    # Merge with admissions
    adm_chunks = pd.read_csv(file_paths['admissions'], usecols=cols_to_use['admissions'], dtype=dtype_spec, chunksize=chunk_size)
    merge_and_save(micro_chunk, adm_chunks, on='subject_id', suffixes=('', '_adm'), output_file=output_file)

    # Merge with diagnoses_icd
    diag_chunks = pd.read_csv(file_paths['diagnoses_icd'], usecols=cols_to_use['diagnoses_icd'], dtype=dtype_spec, chunksize=chunk_size)
    merge_and_save(micro_chunk, diag_chunks, on='subject_id', suffixes=('', '_diag'), output_file=output_file)

    # Merge with labevents
    lab_chunks = pd.read_csv(file_paths['labevents'], usecols=cols_to_use['labevents'], dtype=dtype_spec, chunksize=chunk_size)
    merge_and_save(micro_chunk, lab_chunks, on='subject_id', suffixes=('', '_lab'), output_file=output_file)

    # Merge with procedures_icd
    proc_chunks = pd.read_csv(file_paths['procedures_icd'], usecols=cols_to_use['procedures_icd'], dtype=dtype_spec, chunksize=chunk_size)
    merge_and_save(micro_chunk, proc_chunks, on='subject_id', suffixes=('', '_proc'), output_file=output_file)

    # Merge with patients
    pat_chunks = pd.read_csv(file_paths['patients'], usecols=cols_to_use['patients'], dtype=dtype_spec, chunksize=chunk_size)
    merge_and_save(micro_chunk, pat_chunks, on='subject_id', suffixes=('', '_pat'), output_file=output_file)

    # Merge with icustays
    icu_chunks = pd.read_csv(file_paths['icustays'], usecols=cols_to_use['icustays'], dtype=dtype_spec, chunksize=chunk_size)
    merge_and_save(micro_chunk, icu_chunks, on='subject_id', suffixes=('', '_icu'), output_file=output_file)

    # After each step, the merged data is saved in `final_merged_output.csv`


TREINO DO MODELO PREDITIVO

In [ ]:
import tensorflow as tf
import tensorflow_recommenders as trfs

SUGESTAO 

In [66]:
def drug_suggestion(org_name, resistance):
    if resistance == 'R':
        return "Sugestão de antibiótico alternativo"
    else:
        return "Sugestão de antibiótico padrão"

merged_data['sugestao'] = merged_data.apply(lambda x: drug_suggestion(x['org_name'], x['interpretation']), axis=1)


FERRAMENTA VALIDAÇÃO

In [ ]:
from sklearn.metrics import accuracy_score

# Testar a ferramenta em um conjunto de dados de validação
resultados_reais = merged_data1['antibiotico_real']
sugestoes = merged_data1['sugestao']

acuracia = accuracy_score(resultados_reais, sugestoes)
print(f"Acurácia da ferramenta: {acuracia}")
